In [1]:
import storage as storage
import os
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="XXX",
  database="forestry"
)

mycursor = mydb.cursor()

def getCountries(top_three):
    country1 = None
    country2 = None
    country3 = None
    if len(top_three) == 1:
        country1 = top_three[0]
    if len(top_three) == 2:
        country2 = top_three[1]
    elif len(top_three) == 3:
        country3 = top_three[2]
        
    return country1, country2, country3

def getFirst(lst):
    result = None
    if len(lst) > 0:
        result = lst[0]
    return result

def insertPaper(paper):
    sql_insert = "INSERT INTO PolicyPapers " \
        "(filename, country1, country2, country3, " \
        "date, title, extractedText, cleanedText)" \
        "VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"

    country1, country2, country3 = getCountries(paper.country[:3])

    val = (paper.filename, country1, country2, country3, getFirst(paper.date),
        getFirst(paper.title), ' '.join(paper.extractedText), None)
    mycursor.execute(sql_insert, val)

    mydb.commit()

def inDatabase(filename):
    mycursor.execute("SELECT filename FROM PolicyPapers WHERE filename='%s'" % filename)
    return mycursor.fetchone()

In [7]:
# Might need to generalize multiple levels of directories
def extractPolicyPapersFromDirectory(directory_path):    
    # Retrieve file names inside folder
    files = os.listdir(directory_path)
    print("Folder now open.")
 
    for file in files:
        if file == ".DS_Store":
            continue

        file_path = directory_path + "/" + file
        if not inDatabase(storage.getFileName(file_path)):
            print("PROCESSING", file_path)
            paper = storage.PolicyPaper(file_path)
            insertPaper(paper)
    print("EXTRACTION COMPLETE")
    
extractPolicyPapersFromDirectory("sample data/Kenya")

Folder now open.
EXTRACTION COMPLETE
